## Introduction.

I derived the Black-Scholes formula for European style vanilla FX options in a previous post [here](https://quantophile.github.io/mathsummaries/post/2023/11/19/garman-kohlhagen-for-european-vanilla-fx-options/). The Black-Scholes model $Bl(S_0,K,T,r_{DOM},r_{FOR},\sigma)$ equipped with a single flat volatility parameter $\sigma$ produces option prices which are **NOT consistent** with the observed market prices of FX options across different strikes and maturities. 

Although, the BS model suffers many flaws, it is still often used, at least for quoting purposes. Since all of the other inputs into the model - market data variables such as the stock price $S_0$, the domestic depo rate $r_{DOM}$, the foreign depo rate $r_{FOR}$, and the parameters such as option strike $K$, the time-to-maturity $T$, can be either seen in the market or are known constants, we can easily solve for the value $\sigma_{\text{imp}}$ of the parameter $\sigma$ such that:

$$Bl(S_0,K,T,r_{DOM},r_{FOR},\sigma_{\text{imp}}) = V_{\text{market}}$$

This value $\sigma_{\text{imp}}$ implied from the market price of the option is called the **implied volatility**. 

Thus, although the BS model suffers from flaws, it is mainly used as a quote converter. In the FX options market, option prices are quoted in terms of implied volatilities. The BS formula is used to convert implied vols $\sigma_{\text{imp}}$ to prices and vice versa. The delta hedge to be exchanged between counterparties is calculated according to the BS formula, and this is also true for the Vega hedge of various exotic options. In many cases, the model is also used to run trading books. 


In this note, I explore various delta conventions and derive the greeks. Check out [FX Vol smile](https://mathfinance.com/wp-content/uploads/2017/06/CPQF_Arbeits20_neu2.pdf) by Wyestup! The entire concept of the FX volatility smile is based on the parametrization with respect to delta.

## Quote style conversions.

In FX markets, options are quoted in one of 4 quote styles - domestic per foreign (d/f), percentage foreign (%f), percentage domestic (%d) and foreign per domestic (f/d). 

The standard Black-Scholes formula is:

$$
\begin{align*}
V_{d/f} &= \omega [S_0 e^{-r_{FOR} T} \Phi(d_{+}) - K e^{-r_{DOM}T} \Phi(d_{-})\\
&= \omega e^{-r_{DOM}T}[F \Phi(d_{+}) - K  \Phi(d_{-})]
\end{align*}
$$

### Implementing the Bl Calculator and Option Greeks.


In [ ]:
import numpy as np
from scipy.stats import norm
from enum import Enum
import datetime as dt

class CallPut(Enum):
    CALL_OPTION = 1
    PUT_OPTION = -1

class BlackCalculator:
    """Implements the Black formula to price a vanilla option"""
    
    def at_the_money_forward(
        self,
        s_t : float,
        strike : float,
        now : dt.datetime,
        expiry : dt.datetime,
        r_dom : float,
        r_for : float,
        sigma : float
    ) -> float :
        """Computes the at-the-money forward"""

        foreignDF = np.exp(-r_FOR*(T-t))
        domesticDF = np.exp(-r_DOM*(T-t))
        fwdPoints = foreignDF / domesticDF
        F = fwdPoints * S_t
        return F
            
    def d_plus(S_t,K,t,T,r_DOM,r_FOR,sigma):
        F = at_the_money_forward(S_t,K,t,T,r_DOM,r_FOR,sigma)
        return (np.log(F/K) + (T-t)*(sigma**2)/2)/(sigma * np.sqrt(T - t))

    def d_minus(S_t,K,t,T,r_DOM,r_FOR,sigma):
        F = at_the_money_forward(S_t,K,t,T,r_DOM,r_FOR,sigma)
        return (np.log(F/K) - (T-t)*(sigma**2)/2)/(sigma * np.sqrt(T - t))

    def pv(S_t,K,t,T,r_DOM,r_FOR,sigma, CCY1Notional,callPut):
        F = at_the_money_forward(S_t,K,t,T,r_DOM,r_FOR,sigma)
        omega = callPut.value
        d_plus = dPlus(S_t,K,t,T,r_DOM,r_FOR,sigma)
        d_minus = dMinus(S_t,K,t,T,r_DOM,r_FOR,sigma)
        domesticDF = np.exp(-r_DOM*(T-t))
        
        undiscountedPrice = omega* (F * norm.cdf(omega * d_plus) - K * norm.cdf(omega * d_minus))
        pv = domesticDF * undiscountedPrice * CCY1Notional
        return pv